<h1 align="center"> pyLDPC Tutorial: Sound </h1>

## update:  02/25/16 - v.0.7.0

<b><font color="red"> Since version 0.7: Coding and decoding functions take tG (Transposed G) instead of G, the coding matrix. Functions that construct it (CodingMatrix and CodingMatrix_systematic) return tG instead of G as well. </font></b> 
<br> 

<font color=#0101DF><b> Note: </b> </font> 

 <b> Github doesn't support HTML audio in jupyter's cells, I invite you to open this notebook in <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Sound.ipynb?flush_cache=true"> nbviewer </a> </b> 


This notebook introduces a user's guide of pyLDPC's sub-module Sound: ldpc_sound.
If you would like to know what each function does and go into the construction details, go to <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Sound-Construction.ipynb?flush_cache=true/"> LDPC-Sound Construction Details</a>

First, before anyone sets high expectations, keep in mind that most audio files have a sampling rate of <b>44100 Hz.</b> Which means that decoding only one second of a track is the same as decoding an array of 44100 numbers. Each number has an int16 format, which (as you may have seen in the link above) is transformed to a 17 bits binary array. And that's <u><b>a lot</b></u> to code and decode !

Usually, <b><font color=#A44057> coding plus 1 iteration of decoding of a 5 seconds-track last for about half an hour ! </font></b>

If you want to know why aren't we using low sampling frequencies and make everybody happy, <a href="https://en.wikipedia.org/wiki/44,100_Hz" > this</a> might give you a satisfying answer. (This is a result of <i>Shannon's theroem</i> and I'm not really an expert of the subject ..)

In this tutorial, I'll be using <b><u> wav</u></b> audio files because they're easy to read as numpy arrays with <b><u>scipy</u></b>. 

In [1]:
import numpy as np
import pyldpc
from pyldpc import ldpc_sound
from time import time
from scipy.io import wavfile

## Introduction

Let's code/decode a 5 seconds piano track. 
As mentioned above, coding this file and one iteration of decoding took 30-40mn. If you'd like to impress your friends and collegues by decoding a noisy version of the latest album of Eminem, you would probably have to run the algorithm for <b> days </b>. That's simply because most audio files are recorded with a sampling rate (or frequency if you prefer) of 44.1 kHz. Which means that if you read a wav file with the function <i> wavfile.read() </i> you will get the tuple <i> frequency, array </i>. 

The array is, as explained above, a series of int16 numbers. Some files contain multiple series of numbers (like the one below). We call them <i> channels </i>. The additional channels are not necessary: they provide a stereo-effect. That's why, we'll code and decode only one channel (they're the same, but with a sort of delay in time, so that the sound seems to be coming from different directions). Here's the first channel for example of out piano track that lasts for 5.33 seconds: 
<img src="Equations/SoundT1.png">

<br>

Now you realize how big a very short audio file can be ! Each number is an int16 number which means we'll need 17 bits to binarize it... (remember that an int16 can be negative, so it's included in a 2^16 width segment). 



## PHASE  1:  Evaluate decoding time

This is important because depending on the coding/decoding matrices you use, decoding time can change substantially. 
To have an idea about how much time this is going to take, we'll need to read the file first. 
I chose to use this sample that I found on <a href="http://www.freesound.org"> freesound. </a>

<br>

<audio controls src="Sound/piano/piano.wav" type=audio/wav> </audio>

<br>

> ### Step 1 :  Put it in numbers

In [2]:
freq, piano = wavfile.read("Sound/piano/piano.wav")
print("Frequency: {} samples/sec".format(freq))
print("\nPiano Array:\n",piano)
print("\nPiano's shape:",piano.shape)
print("\nPiano's type:",piano.dtype)
print("\nPiano's duration: {} seconds.".format(round(len(piano)/freq)))

Frequency: 44100 samples/sec

Piano Array:
 [[   0    0]
 [   0    0]
 [   9  766]
 ..., 
 [2024 3144]
 [2046 3960]
 [ -29  -66]]

Piano's shape: (235200, 2)

Piano's type: int16

Piano's duration: 5 seconds.


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy-0.17.0-py3.4-macosx-10.6-intel.egg/scipy/io/wavfile.py:221: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


So our audio file has a standard 44.1 kHz sampling rate and lasts for 5 seconds. Let's code and decode a very small part of it: let's say <b>11 025 samples</b>, which corresponds exactly to a <b>quarter of a second !</b> 

This coding & decoding test will give us an idea about how much time decoding the rest of the file would take.

> ### Step 2: Generate coding & decoding matrices with k = 17

I propose the following combination. Keep in mind that changing the three numbers n, d_v, d_c will change your decoding time. Hence, you must keep the same matrices later on phase two.

In [3]:
test = piano[:freq//4]
wavfile.write("Sound/piano/test/test.wav",freq,test)

the test file lasts for 250 ms as mentioned above .. there is not much to hear. 

<br>

<audio controls src="Sound/piano/test/test.wav" type=audio/wav> </audio>

In [4]:
n = 60
d_v = 3
d_c = 4

H = pyldpc.RegularH(n,d_v,d_c) 
H,tG = pyldpc.CodingMatrix_systematic(H) #Must be systematic to visualize the noisy track 
k = tG.shape[1]
k 

17

> ### Step 3: Binarize the sample and code it

Binarize the test array with the function:

```python
ldpc_sound.Audio2Bin (audio array) -> return (audio binary form)
```

Then code it with the function:

```python
ldpc_sound.SoundCoding (G coding matrix, audio binary form, snr) -> return (coded audio, noisy audio) 
```

We choose to code it with SNR = 6 decibels (which corresponds to a standard-deviation around 0.45). 


In [6]:
snr = 6
t = time()
test_bin = ldpc_sound.Audio2Bin(test) 
test_coded, test_noisy = ldpc_sound.SoundCoding(tG,test_bin,snr)
t = time() - t
print("Coding time of {} samples: {}mn {}s\n".format(len(test), int(t//60),int(t%60)))
wavfile.write("Sound/piano/test/test_noisy.wav",freq,test_noisy[:freq//4])

Coding time of 11025 samples: 0mn 0s



Here's what the noisy test sounds like:

<br>

<audio controls src="Sound/piano/test/test_noisy.wav" type=audio/wav></audio>

<br>

> ### Step 4: Decode the coded sample

Now decode it with the function:

```python
ldpc_sound.SoundDecoding
```

We'll first 

In [7]:
max_iter = 1
t = time()
test_decoded = ldpc_sound.SoundDecoding(tG,H,test_coded,snr,max_iter)
t = time() - t 
wavfile.write("Sound/piano/test/test_decoded.wav",freq,test_decoded)
print("Decoding time of {} samples and {} max iterations: {}mn {}s\n".format(len(test),max_iter, int(t//60),int(t%60)))
print("Bit Error Rate in %: ",100*ldpc_sound.BER_audio(test_bin,ldpc_sound.Audio2Bin(test_decoded)),"%")

Decoding time of 11025 samples and 1 max iterations: 0mn 46s

Bit Error Rate in %:  0.0682939842604 %


Here's what the decoded test sounds like:

<br>

<audio controls src="Sound/piano/test/test_decoded.wav" type=audio/wav></audio>

<br>

We'll need to increase max_iter for a better decoding:

In [10]:
max_iter = 10
t = time()
test_decoded = ldpc_sound.SoundDecoding(tG,H,test_coded,snr,max_iter)
t = time() - t 
wavfile.write("Sound/piano/test/test_decoded.wav",freq,test_decoded)
print("Decoding time of {} samples and {} max iterations: {}mn {}s\n".format(len(test),max_iter, int(t//60),int(t%60)))
print("Bit Error Rate in %: ",100*ldpc_sound.BER_audio(test_bin,ldpc_sound.Audio2Bin(test_decoded)),"%")

Decoding time of 11025 samples and 10 max iterations: 0mn 49s

Bit Error Rate in %:  0.0 %


Let's say coding-decoding <b> one quarter of a second </b> takes <b>1mn</b>. Then coding-decoding <b> one second (44 100 numbers) </b> should take around <b>4-5mn</b>. Decoding the whole 5 seconds track will therefore take <b> 20 mn </b>. Keep in mind that this is an only an estimation that can be effected with the properties of coding and decoding matrices.

> You're probably wondering: <b>Why</b> doesn't he code and decode entirely the audio file and just waits for it to be done ? 

Of course you can do that. I even encourage that for such short audio files. But what if you would like to simulate a transmission of an <b> entire song </b>? It could take days. That's why I prefer to code-encode second by second and then concatenate the audio files as we'll see later. That way, I know what was coded and wasn't coded yet, and more importantly coded and decoded files are locally saved. And I can test whether my Bit Error Rate is satsifying (then I should go on with decoding the rest) or BER is too high (so I'll have to increase max_iter). Besides waiting for a program for hours can make you very uncomfortable: you have no idea whether it's <i>actually working</i> or not. 

## PHASE 2: Schedule your coding & decoding  ! 


> <h4> Main Idea: </h4> <font color=#A44057><b> The idea I'm trying to express here is, instead of coding and decoding the whole track at once, let's cut it into smaller parts that can be coded & decoded in one or two hours, and for each part, code it & decode it second by second in a loop. The purpose of the loop is to save each 1-second noisy and decoded track. </b></font>


### Step 1: Structure your saving folders

In this case, we'll have to realize 5 code-decode operations of 8 minutes each. Since the operation only takes 40 mn, we're going to carry out the 5 operations one after another in one loop. In case you would like to code and decode longer tracks, you'll probably need to increase the number of cuts, and repeat the process. 

I created 3 folders locally in each max_iter=i folder. (For now, max_iter = 1, I'll create more if needed).
```python
Sound/piano/snr=6
                 /max_iter=1
                      /samples
                      /noisy
                      /decoded
          /test/
            test_decoded.wav
            test_noisy.wav
            test.wav
          piano.wav
```
The 3 folders above will contain the 1-second tracks: original, noisy and decoded. Then we'll concatenate the files of each folder to get the whole audio file in the three forms. (Except the files in samples, we don't need to gather 1-seconds original tracks since we've got the original wavfile.)

We're going to code and decode one second at a time and save noisy and decoded parts. which will give us an idea about how good is decoding is and perhaps anticipate making more decoding iterations by increasing max_iter. 

### Step 2: Read, binarize the file and measure its duration

In [11]:
freq, piano = wavfile.read("Sound/piano/piano.wav")
length = len(piano)
duration = int(length/freq)
piano_bin = ldpc_sound.Audio2Bin(piano)

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy-0.17.0-py3.4-macosx-10.6-intel.egg/scipy/io/wavfile.py:221: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Since the whole coding-decoding operation lasts only for 15-20 mn, we can make it all in one loop. Let's see how accurate our decoding is using only 1 iteration of the BP full-log algorithm. 

We'll simulate a noisy channel with S.N.R = 6 db (<i> Signal Noise Ratio </i>, 6db corresponds to a standard-deviation 0.35 )

### Step 3: Code and decode:

In [13]:
snr = 6
max_iter = 10

for second in range(duration+1):
    
    sample = piano[second*freq:min((second+1)*freq,length-1)]
    wavfile.write("Sound/piano/snr="+str(snr)+"/max_iter="+str(max_iter)+"/samples/"+str(second)+".wav",freq,sample)
    
    sample_bin = piano_bin[second*freq:(second+1)*freq]

    tc = time()
    coded,noisy= ldpc_sound.SoundCoding(tG,sample_bin,snr)
    tc = time()-tc
    
    wavfile.write("Sound/piano/snr="+str(snr)+"/max_iter="+str(max_iter)+"/noisy/"+str(second)+".wav",freq,noisy)

    print("Coding time of the second {}: tc = {}mn {}s\n".format(second, int(tc//60),int(tc%60)))

    td = time()
    decoded = ldpc_sound.SoundDecoding(tG,H,coded,snr,max_iter)   
    td = time()-td
    
    wavfile.write("Sound/piano/snr="+str(snr)+"/max_iter="+str(max_iter)+"/decoded/"+str(second)+".wav",freq,decoded)
    
    print("Decoding time of the second {} and max iterations ={} : td = {}mn {}s\n".format(second,max_iter, int(td//60),int(td%60)))
    
    

Coding time of the second 0: tc = 0mn 2s

Decoding time of the second 0 and max iterations =10 : td = 3mn 13s

Coding time of the second 1: tc = 0mn 2s

Decoding time of the second 1 and max iterations =10 : td = 3mn 0s

Coding time of the second 2: tc = 0mn 2s

Decoding time of the second 2 and max iterations =10 : td = 2mn 56s

Coding time of the second 3: tc = 0mn 2s

Decoding time of the second 3 and max iterations =10 : td = 2mn 55s

Coding time of the second 4: tc = 0mn 2s

Decoding time of the second 4 and max iterations =10 : td = 2mn 54s

Coding time of the second 5: tc = 0mn 0s

Decoding time of the second 5 and max iterations =10 : td = 0mn 57s



### Step 4: Concatenate the 1 second tracks:

Now let's concatenate the arrays of each folder:

To help you understand the cell below, you need to know that it's os.walk takes a path as an argument and generates the tuple: path, dirs, elements:
- path: path of the argument 
- dirs: list of the folders contained in the argument
- elements: list of the files contained in the argument

The argument here is <b><u> the folder noisy </u></b> (in the first loop) and <b><u> the folder decoded </u></b> (in the second loop). Only <i> elements </i> is useful to concatenate the wav files of each folder. 

In [14]:
import os 
folder_source = "Sound/piano/snr="+str(snr)+"/max_iter="+str(max_iter)
folder_noisy = folder_source + "/noisy"
folder_decoded = folder_source + "/decoded"

piano_noisy = np.array([],dtype=np.int16)
for path,dirs,elements in os.walk(folder_noisy):
        for track in elements:
            if "wav" in track.split(".") and not "all" in track.split("."):
                piano_noisy = np.concatenate((piano_noisy,wavfile.read(folder_noisy+"/"+track)[1]))

wavfile.write(folder_noisy+"/all.wav",freq,piano_noisy)

piano_decoded = np.array([],dtype=np.int16)
for path,dirs,elements  in os.walk(folder_decoded):
        for track in elements:
            if "wav" in track.split(".") and not "all" in track.split("."):
                piano_decoded = np.concatenate((piano_decoded,wavfile.read(folder_decoded+"/"+track)[1]))

wavfile.write(folder_decoded+"/all.wav",freq,piano_decoded)

### Step 5: Evaluate the quality of your decoding

Let's compute Bit Error Rate using the function 

```python
ldpc_sound.BER_audio
```
It compares the original binary audio and the binary decoded one bit by bit and returns the probability of error. 

In [15]:
freq_decoded, decoded = wavfile.read("Sound/piano/snr="+str(snr)+"/max_iter="+str(max_iter)+"/decoded/all.wav")
decoded_bin = ldpc_sound.Audio2Bin(decoded)
piano_bin = ldpc_sound.Audio2Bin(piano[:,0])

BER_piano = ldpc_sound.BER_audio(decoded_bin,piano_bin)

print("Bit Error Rate in %:",100*BER_piano,"%")

Bit Error Rate in %: 7.50300120048e-05 %


<h4> Tada ! Our decoding is almost error-free ! No need to add more iterations ! </h4>

Let's have a look at our fully concatenated 5-seconds tracks 

The original track:

<audio controls src="Sound/piano/piano.wav" type=audio/wav></audio>

The noisy track:

<audio controls src="Sound/piano/snr=9/max_iter=1/noisy/all.wav" type=audio/wav></audio>

The decoded track:

<audio controls src="Sound/piano/snr=9/max_iter=1/decoded/all.wav" type=audio/wav></audio>




### Step 6: Increase max_iter if needed

Depending on the channel model you built (SNR's value) and the quality of decoding you're hoping for, you may probably still have a little noisy sounds (BER > 0) after only one iteration of decoding. Sometimes, usually with small SNRs (less than 6), you may even need to complete hundreds of iterations to get a satisfying results. The idea here is making 2 iterations instead of one won't make a valuable difference. Personally, I increase max_iter like this :
1, 15, 50, 100, 200. However, for low values of SNR, perfect decoding may not be possible no matter how you increase the number of iterations: the noise is in this case too loud to re-establish 100% of the information.

<font color=#A44057> <h2> Application to songs </h2> </font>

As I said before, you can use the same method described in this tutorial by first cutting your 3 minutes track in many (too many) 5 seconds tracks. And then concatenate the whole mess ! You will need to make sure that no file gets overwritten ! 

As soon as the first song is coded/decoded, an example will be added in this section.